In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from extract import LOCM3,LOCM2,LOCM
from utils import read_plan, read_json_file
from evaluator import ExecutabilityEvaluator
from collections import defaultdict
import json

In [2]:
def test_locm3_acceptance(domain_name,train_traces, test_traces, invalid_suffixes, arity=1):
    debug = {}
    try:
        extractor = LOCM3(event_arity=arity, debug=debug)
        model = extractor.extract_model(train_traces)
        learned_domain = model.to_pddl_domain(domain_name)
        
        evaluator = ExecutabilityEvaluator(learned_domain, debug=False)
        valid_res = []
        invalid_res = []
        for problem, trace in test_traces.items():
            valid_acceptance, invalid_acceptance = evaluator.get_acceptance_rate(trace, invalid_suffixes[problem])
            valid_res.append(valid_acceptance)
            invalid_res.append(invalid_acceptance)
        if len(valid_res) == 0:
            valid = 0
        else:
            valid = sum(valid_res) / len(valid_res)
        if len(invalid_res) == 0:
            invalid = 0
        else:
            invalid = sum(invalid_res) / len(invalid_res)
        return valid, invalid
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0,1


def test_locm2_acceptance(domain_name,train_traces, test_traces, invalid_suffixes):
    debug = {}
    try:
        locm2 = LOCM2(state_param=True, viz=False, debug=debug)
        model = locm2.extract_model(train_traces)
        learned_domain = model.to_pddl_domain(domain_name)
        gt_filename = os.path.join('../../data', 'goose-benchmarks', 'tasks', domain_name, 'domain.pddl')
        
        evaluator = ExecutabilityEvaluator(learned_domain, gt_filename, debug=False)
        valid_res = []
        invalid_res = []
        for problem, trace in test_traces.items():
            valid_acceptance, invalid_acceptance = evaluator.get_acceptance_rate(trace, invalid_suffixes[problem])
            valid_res.append(valid_acceptance)
            invalid_res.append(invalid_acceptance)
        if len(valid_res) == 0:
            valid = 0
        else:
            valid = sum(valid_res) / len(valid_res)
        if len(invalid_res) == 0:
            invalid = 0
        else:
            invalid = sum(invalid_res) / len(invalid_res)
        return valid, invalid
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0,1
    

def test_locm_acceptance(domain_name,train_traces, test_traces, invalid_suffixes, arity=1, state_param=True):
    debug = {}
    try:
        locm = LOCM(event_arity=arity, state_param=state_param, viz=False, debug=debug)
        model = locm.extract_model(train_traces)
        learned_domain = model.to_pddl_domain(domain_name)
        gt_filename = os.path.join('../../data', 'goose-benchmarks', 'tasks', domain_name, 'domain.pddl')
        
        evaluator = ExecutabilityEvaluator(learned_domain, gt_filename, debug=False)
        valid_res = []
        invalid_res = []
        for problem, trace in test_traces.items():
            valid_acceptance, invalid_acceptance = evaluator.get_acceptance_rate(trace, invalid_suffixes[problem])
            valid_res.append(valid_acceptance)
            invalid_res.append(invalid_acceptance)
        if len(valid_res) == 0:
            valid = 0
        else:
            valid = sum(valid_res) / len(valid_res)
        if len(invalid_res) == 0:
            invalid = 0
        else:
            invalid = sum(invalid_res) / len(invalid_res)
        return valid, invalid
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0,1

In [3]:
plain_traces = defaultdict(lambda: defaultdict())
with open("../../data/plain_traces/plain_traces.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        details = line.split("&&")

        domain_name = details[0]
        problem_name = details[2]
        plan = details[-1]

        plain_traces[domain_name][problem_name]= read_plan(plan)


In [4]:
train_traces = defaultdict(list)
data = read_json_file("../../data/training_data/traces_plan_po_r10.json")
for learning_obj in data:
    domain_name = learning_obj["domain"]
    train_traces[domain_name].append(learning_obj)
    


In [5]:
invalid_suffixes = defaultdict(lambda: defaultdict(list))

with open("../../data/plain_traces/invalid_suffixes.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        details = line.split("&&")
        domain_name = details[0]
        problem_name = details[1]
        plan = details[2]
        
        invalid_suffixes[domain_name][problem_name].append(read_plan(plan)[0])


# LOCM2 Arity1

In [6]:
with open("./locm2_a1_acceptance.csv", "w") as f:
    f.write("ID, Domain, len, len%, acceptance_rate, invalid_acceptance_rate\n")
    for domain, items in train_traces.items():
        print(f"Testing domain: {domain}")
        for item in items:
            traces = item["traces"]
            problem_name
            try:
                rate, invalid_rate = test_locm2_acceptance(domain, traces, plain_traces[domain], invalid_suffixes[domain])
            except Exception as e:
                print(f"Error in balanced executability for domain {domain}: {e}")
                continue

        # Test balanced executability
            f.write(f"{item['id']}, {domain}, {item['total_length']}, {item['len%']}, {rate}, {invalid_rate}\n")
            f.flush()
            print(f"Balanced Executability for {domain}: {(rate, invalid_rate)}")

Testing domain: blocksworld
Error processing domain blocksworld: 'tuple' object has no attribute 'copy'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'copy'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'copy'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'copy'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'copy'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'copy'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'copy'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'copy'
Balanced Executability for blockswo

# LOCM Arity1 with state params

In [7]:
with open("./locm_a1_with_param_acceptance.csv", "w") as f:
    f.write("ID, Domain, len, len%, valid_acceptance, invalid_valid_acceptance\n")
    for domain, items in train_traces.items():
        print(f"Testing domain: {domain}")
        for item in items:
            traces = item["traces"]
            problem_name
            try:
                rate, invalid_rate = test_locm_acceptance(domain, traces, plain_traces[domain], invalid_suffixes[domain])
            except Exception as e:
                print(f"Error in balanced executability for domain {domain}: {e}")
                continue

        # Test balanced executability
            f.write(f"{item['id']}, {domain}, {item['total_length']}, {item['len%']}, {rate}, {invalid_rate}\n")
            f.flush()
            print(f"Balanced Executability for {domain}: {(rate, invalid_rate)}")

Testing domain: blocksworld
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blockswo

# LOCM Arity2 w/o state params

In [8]:
with open("./locm_a1_wo_param_acceptance.csv", "w") as f:
    f.write("ID, Domain, len, len%, valid_acceptance, invalid_valid_acceptance\n")
    for domain, items in train_traces.items():
        print(f"Testing domain: {domain}")
        for item in items:
            traces = item["traces"]
            problem_name
            try:
                rate, invalid_rate = test_locm_acceptance(domain, traces, plain_traces[domain], invalid_suffixes[domain], arity=2, state_param=False)
            except Exception as e:
                print(f"Error in balanced executability for domain {domain}: {e}")
                continue

        # Test balanced executability
            f.write(f"{item['id']}, {domain}, {item['total_length']}, {item['len%']}, {rate}, {invalid_rate}\n")
            f.flush()
            print(f"Balanced Executability for {domain}: {(rate, invalid_rate)}")

Testing domain: blocksworld
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'tuple' object has no attribute 'name'
Balanced Executability for blockswo

# LOCM3 Arity2

In [9]:
with open("./locm3_a2.csv", "w") as f:
    f.write("ID, Domain, len, len%, acceptance_rate, invalid_acceptance_rate\n")
    for domain, items in train_traces.items():
        print(f"Testing domain: {domain}")
        for item in items:
            traces = item["traces"]
            problem_name
            try:
                rate, invalid_rate = test_locm3_acceptance(domain, traces, plain_traces[domain], invalid_suffixes[domain], arity=2)
            except Exception as e:
                print(f"Error in balanced executability for domain {domain}: {e}")
                continue

        # Test balanced executability
            f.write(f"{item['id']}, {domain}, {item['total_length']}, {item['len%']}, {rate}, {invalid_rate}\n")
            f.flush()
            print(f"Balanced Executability for {domain}: {(rate, invalid_rate)}")

Testing domain: blocksworld
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
Balanced Executability for blocksworld: (0.03571428571428571, 0.0)
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action foun

# LOCM3 Arity1

In [10]:
with open("./locm3_a1.csv", "w") as f:
    f.write("ID, Domain, len, len%, acceptance_rate, invalid_acceptance_rate\n")
    for domain, items in train_traces.items():
        print(f"Testing domain: {domain}")
        for item in items:
            traces = item["traces"]
            problem_name
            try:
                rate, invalid_rate = test_locm3_acceptance(domain, traces, plain_traces[domain], invalid_suffixes[domain])
            except Exception as e:
                print(f"Error in balanced executability for domain {domain}: {e}")
                continue

        # Test balanced executability
            f.write(f"{item['id']}, {domain}, {item['total_length']}, {item['len%']}, {rate}, {invalid_rate}\n")
            f.flush()
            print(f"Balanced Executability for {domain}: {(rate, invalid_rate)}")

Testing domain: blocksworld
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
Balanced Executability for blocksworld: (0.03571428571428571, 0.0)
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action found for pickup
no action foun